In [153]:
import pandas as pd
import numpy as np
import re

change everything to lowercase
If ingredient contains 'vodka' change to vodka
If ingredient contains 'gin' change to gin
Take a closer at other liquors
Maybe remove words like fresh?
Check ingredient values:
     - bitters : change measure to 0.1 
     - looks for incorrect recipes... such as Eggnog Supreme
     - etc ...

pivot table
Remove duplicates (martini had 4 versions of the same thing)
Drop drinks with 1 ingredient

In [154]:
def replace_gin(ingredient):
    if re.search(r'\bgin\b', ingredient, re.IGNORECASE):
        return 'gin'
    else:
        return ingredient

In [155]:
url = "https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-05-26/boston_cocktails.csv"
df = pd.read_csv(url)
df

,name,category,row_id,ingredient_number,ingredient,measure
0,Gauguin,Cocktail Classics,1,1,Light Rum,2 oz
1,Gauguin,Cocktail Classics,1,2,Passion Fruit Syrup,1 oz
2,Gauguin,Cocktail Classics,1,3,Lemon Juice,1 oz
3,Gauguin,Cocktail Classics,1,4,Lime Juice,1 oz
4,Fort Lauderdale,Cocktail Classics,2,1,Light Rum,1 1/2 oz
...,...,...,...,...,...,...
3638,Vow Of Silence,Gin,990,2,Grapefruit Juice,3/4 oz
3639,Vow Of Silence,Gin,990,3,Fresh Lime Juice,1/2 oz
3640,Vow Of Silence,Gin,990,4,Yellow Chartreuse,1/2 oz
3641,Vow Of Silence,Gin,990,5,Amaro,1/4 oz


In [156]:
df.shape

(3643, 6)

In [157]:
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].str.lower()

df.head(25)

,name,category,row_id,ingredient_number,ingredient,measure
0,gauguin,cocktail classics,1,1,light rum,2 oz
1,gauguin,cocktail classics,1,2,passion fruit syrup,1 oz
2,gauguin,cocktail classics,1,3,lemon juice,1 oz
3,gauguin,cocktail classics,1,4,lime juice,1 oz
4,fort lauderdale,cocktail classics,2,1,light rum,1 1/2 oz
5,fort lauderdale,cocktail classics,2,2,sweet vermouth,1/2 oz
6,fort lauderdale,cocktail classics,2,3,juice of orange,1/4 oz
7,fort lauderdale,cocktail classics,2,4,juice of a lime,1/4 oz
8,apple pie,cordials and liqueurs,3,1,apple schnapps,3 oz
9,apple pie,cordials and liqueurs,3,2,cinnamon schnapps,1 oz


In [158]:
df.groupby('name')['ingredient'].nunique()

name
1626                      5
19th century              4
a. j.                     2
absinthe cocktail         4
absinthe drip cocktail    2
                         ..
yokahama romance          4
yolanda                   5
zaza cocktail             2
zero mint                 1
zombie                    6
Name: ingredient, Length: 989, dtype: int64

In [159]:
df = df.loc[df['category'] != 'Non-alcoholic Drinks']

In [160]:
df.loc[df['ingredient'] == 'bitters', 'measure'] = 0.1

In [161]:
# grouped = df.groupby('name')['ingredient_number'].sum().sort_values().head(500)
# grouped.value_counts()
grouped = df.groupby('name').size()
grouped.value_counts().sort_values()

1     31
6     57
2    118
5    194
3    292
4    297
dtype: int64

In [162]:
df = df.groupby('name').filter(lambda x: len(x) > 1)
df.shape

(3612, 6)

In [163]:
ingredient_counts = df.groupby('name').size()
exactly_two_ingredients = ingredient_counts == 2
df_two_ingredients = df[df['name'].isin(exactly_two_ingredients[exactly_two_ingredients].index)]
df_two_ingredients['name'].unique()

array(['apple pie', 'caribbean champagne', 'amaretto stinger',
       'amaretto and cream', 'montreal club bouncer',
       'martini (dry) (5-to-1)', "mamie's sister", 'lone tree cocktail',
       'london buck', 'leap frog highball', 'hoffman house cocktail',
       'white russian', 'greyhound', 'gin and it', 'gin cocktail',
       'flying dutchman', 'fifty-fifty cocktail', 'dry martini (5-to-1)',
       'vodka stinger', 'bulldog highball', 'billy taylor',
       'highland cooler', 'tempter cocktail',
       'merry widow cocktail no. 2', 'hennessy martini', 'cherry sling',
       'cherry fizz', 'vodka cooler', 'carrol cocktail', "bull's eye",
       'dixie julep', "buddy's favorite", 'bourbon and water',
       'sake martini', 'godfather', 'martini', 'mandarine sour',
       'twister', 'harvard cooler', 'grenadine rickey', 't.n.t. no. 1',
       'the debonair', 'screwdriver', 'dry martini', 'dillatini cocktail',
       'white cargo cocktail', 'french quarter', 'cowboy cocktail',
      

In [164]:
df[df['name'] == 'Eggnog Supreme'] # drop this

,name,category,row_id,ingredient_number,ingredient,measure


In [165]:
df['ingredient'] = df['ingredient'].apply(lambda x: 'vodka' if 'vodka' in x.lower() else x)
df['ingredient'] = df['ingredient'].apply(lambda x: 'tequila' if 'tequila' in x.lower() else x)
df['ingredient'] = df['ingredient'].apply(lambda x: 'bitter' if 'bitter' in x.lower() else x)
df['ingredient'] = df['ingredient'].apply(lambda x: 'lime juice' if 'lime juice' in x.lower() else x)
df['ingredient'] = df['ingredient'].apply(lambda x: 'lemon juice' if 'lemon juice' in x.lower() else x)
df['ingredient'] = df['ingredient'].apply(lambda x: 'light cream' if 'light cream' in x.lower() else x)
df['ingredient'] = df['ingredient'].apply(replace_gin)
df['ingredient'] = df['ingredient'].str.replace('fresh', '', case=False).str.strip()
df['ingredient'].unique()

array(['light rum', 'passion fruit syrup', 'lemon juice', 'lime juice',
       'sweet vermouth', 'juice of orange', 'juice of a lime',
       'apple schnapps', 'cinnamon schnapps', 'powdered sugar',
       'dark rum', 'cranberry juice', 'pineapple juice', 'orange curacao',
       'sour mix', 'bourbon whiskey', 'simple syrup', 'soda water',
       'orange and lemon wheels, maraschino cherry',
       'cherry-flavored brandy', 'light cream', 'triple sec',
       'maraschino', 'creme de banana', 'amaretto', 'scotch whiskey',
       'carrot juice', 'tawny port', 'agave nectar', 'bacardi rum',
       'grenadine', 'apple brandy', 'brandy', 'gin', 'anisette',
       'orange juice', 'dry vermouth', 'apricot flavored brandy',
       'white creme de menthe', 'green creme de menthe', 'mint leaves',
       'margarita mix', 'orange half-wheel', 'green chartreuse', 'bitter',
       'straight rye whiskey', 'benedictine', 'honey syrup', 'egg white',
       'irish whiskey', 'ginger ale or soda water',
 

In [166]:
df = df[df['name'] != 'martini (dry) (5-to-1)']
df = df[df['name'] != 'martini (medium)']
df = df[df['name'] != 'dry martini (5-to-1)']
df = df[df['name'] != 'dry martini']
# df = df[df['name'] != 'martini (dry) (5-to-1)']


In [167]:
for i in df['name']:
    if 'martini' in i:
        print(i)

hennessy martini
hennessy martini
sake martini
sake martini
martini
martini
redhead martini
redhead martini
redhead martini
redhead martini
redhead martini
pickled martini
pickled martini
pickled martini
pickled martini
vodka martini
vodka martini
sand-martini cocktail
sand-martini cocktail
sand-martini cocktail
sand-martini cocktail
sake martini (sakini)
sake martini (sakini)
french martini
french martini
french martini
flatiron martini
flatiron martini
flatiron martini
dreamy dorini smoking martini
dreamy dorini smoking martini
dreamy dorini smoking martini
rum martini
rum martini
martinique rose
martinique rose
martinique rose
martinique rose
martinique rose
traditional martini (dry)
traditional martini (dry)
traditional martini (dry)


In [168]:
ingredient_counts = df.groupby('ingredient').size().sort_values(ascending=True)
ingredient_counts

ingredient
honey-currant syrup      1
irish cream liqueur      1
iced tea                 1
hot tea                  1
root beer                1
                      ... 
lime juice             131
vodka                  164
bitter                 166
lemon juice            214
gin                    239
Length: 463, dtype: int64

In [169]:
# Count the number of ingredients for each drink
ingredient_count = df.groupby('name')['ingredient'].count()

# Filter to get only drinks with 1 or 2 ingredients
drinks_with_1_or_2_ingredients = ingredient_count[ingredient_count.isin([1, 2])]
drinks_with_1_ingredients = ingredient_count[ingredient_count.isin([1])]


# Now drinks_with_1_or_2_ingredients contains the names of the drinks and their respective ingredient counts
print(drinks_with_1_or_2_ingredients)

name
a. j.                     2
absinthe drip cocktail    2
amaretto and cream        2
amaretto stinger          2
antoine special           2
                         ..
white lady                2
white russian             2
widow's dream             2
widow's dream cocktail    2
zaza cocktail             2
Name: ingredient, Length: 115, dtype: int64


In [185]:
df['measure'].unique()

array(['2 oz', '1 oz', '1 1/2 oz', '1/2 oz', '1/4 oz', '3 oz', '1 1/4 oz',
       '3/4 oz', '2 1/2 oz', '5 oz', '1', 0.1, '1 dash', '4 oz', '6',
       '1 3/4 oz', '1/2 or 1', '2 1/4 oz', '5 or 6', '6 oz', 'for glass',
       '1/2 tsp', '24 oz', '1c', '1 tsp', '2', '2 750-ml', '25', '5',
       '1 750-ml', '3 slices', '16 oz', '6 fresh', '7', '2 -  3 drops',
       'splash', '12', '1  3/4 oz', '1 splash'], dtype=object)

In [186]:
df

,name,category,row_id,ingredient_number,ingredient,measure
0,gauguin,cocktail classics,1,1,light rum,2 oz
1,gauguin,cocktail classics,1,2,passion fruit syrup,1 oz
2,gauguin,cocktail classics,1,3,lemon juice,1 oz
3,gauguin,cocktail classics,1,4,lime juice,1 oz
4,fort lauderdale,cocktail classics,2,1,light rum,1 1/2 oz
...,...,...,...,...,...,...
3638,vow of silence,gin,990,2,grapefruit juice,3/4 oz
3639,vow of silence,gin,990,3,lime juice,1/2 oz
3640,vow of silence,gin,990,4,yellow chartreuse,1/2 oz
3641,vow of silence,gin,990,5,amaro,1/4 oz


In [206]:
dummies = pd.get_dummies(df['ingredient'])
dummies = pd.concat([df['name'], dummies], axis=1)
dummies = dummies.groupby('name').sum()

In [207]:
# dummies.set_index('name', inplace=True)
dummies.to_csv('cleaned_cocktail_dummies.csv', index=True)